# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import pickle
import requests
import json

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
from train import clean_data

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.core import Pipeline
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.pipeline.steps import AutoMLStep
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.environment import Environment

#pip install pyarrow>=0.12.0 --upgrade

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')

# choose a name for experiment
experiment_name = 'Titanic_project'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

amlcompute_cluster_name = "ML-cluster-uda"

try:
      compute_target=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
      print("Found existing cluster.Using it")
except ComputeTargetException:
      compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
      compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True, min_node_count=1, timeout_in_minutes=10)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ARHCZABXR to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-136918
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-136918
Found existing cluster.Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
found = False
key = "titanic"
description_text = "Titanic survival classification data from Kaggle"

if key in ws.datasets.keys(): 
        found = True
        data = ws.datasets[key] 
else :
    data = pd.read_csv("titanic.csv")

x, y = clean_data(data.to_pandas_dataframe())
x['y']=y
print(x.head(5))
print(x.describe())

# turning pandas dataframe to Tabular data
data_folder = './data'
os.makedirs(data_folder, exist_ok=True)
local_path = 'data/prepared.csv'
x.to_csv(local_path)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, ('data/prepared.csv'))])

   PassengerId  Pclass  Sex Age  SibSp  Parch     Fare  FamilySize  IsAlone  y
0            1       3    0   1      1      0   7.2500           1        0  0
1            2       1    1   2      1      0  71.2833           1        0  1
2            3       3    1   3      0      0   7.9250           0        1  1
3            4       1    1   4      1      0  53.1000           1        0  1
4            5       3    0   5      0      0   8.0500           0        1  0
       PassengerId      Pclass         Sex       SibSp       Parch  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    2.308642    0.352413    0.523008    0.381594   
std     257.353842    0.836071    0.477990    1.102743    0.806057   
min       1.000000    1.000000    0.000000    0.000000    0.000000   
25%     223.500000    2.000000    0.000000    0.000000    0.000000   
50%     446.000000    3.000000    0.000000    0.000000    0.000000   
75%     668.500000    3.000000    1.

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'Accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = "./data",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             n_cross_validations=5,
                             **automl_settings
                            )


In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=False)

Running on remote.
No run_configuration provided, running on ML-cluster-uda with default configuration
Running on remote compute: ML-cluster-uda
Parent Run ID: AutoML_b459f211-9402-49ae-a196-0bc7d069db8f

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing val

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
best_run_auto, best_model = remote_run.get_output()
best_run_auto.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":5,"CurrentNodeCount":5}',
 'ensembled_iterations': '[1, 6, 13, 7, 26, 34, 15, 33, 8]',
 'ensembled_algorithms': "['XGBoostClassifier', 'ExtremeRandomTrees', 'XGBoostClassifier', 'ExtremeRandomTrees', 'LightGBM', 'LogisticRegression', 'ExtremeRandomTrees', 'SGD', 'RandomForest']",
 'ensemble_weights': '[0.16666666666666666, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]',
 'best_individual_pipeline_score': '0.8523997765446302',
 'best_individual_iteration': '1',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [31]:
description = 'Titanic survival AutoML model'
model = best_run_auto.register_model(model_name='model', description = description, model_path = 'outputs/model.pkl', tags={'area': 'titanic'}, model_framework=Model.Framework.SCIKITLEARN, 
model_framework_version='0.19.1')

print(model)
print(model.name, model.id, model.version, sep='   ')

Model(workspace=Workspace.create(name='quick-starts-ws-136918', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-136918'), name=model, id=model:1, version=1, tags={'area': 'titanic'}, properties={})
model   model:1   1


In [33]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=2,
                                                enable_app_insights=True, # standout suggestion
                                                auth_enabled=True)  

service_name = 'titanic-survival-service'

service = Model.deploy(ws, service_name, [model], deployment_config=aci_config)

service.wait_for_deployment(show_output = True)

print(service.state)
print("scoring URI: " + service.scoring_uri)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................................................................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 170affca-500e-4671-8223-fd4eab4300c6
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'azureml.automl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'azureml.automl', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 170affca-500e-4671-8223-fd4eab4300c6
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'azureml.automl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'azureml.automl', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 170affca-500e-4671-8223-fd4eab4300c6\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"AciDeplo

In [38]:
#VAIHTOEHTO 2, JOKA EI VIELÄ OIKEIN TOIMI

myenv = Environment.get(workspace=ws, name="AzureML-Tutorial")
print(myenv.python.conda_dependencies.serialize_to_string())

channels:
- anaconda
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.21.0.post1
  - azureml-defaults==1.21.0
  - azureml-telemetry==1.21.0
  - azureml-train-restclients-hyperdrive==1.21.0
  - azureml-train-core==1.21.0
  - azureml-widgets==1.21.0
  - azureml-pipeline-core==1.21.0
  - azureml-pipeline-steps==1.21.0
  - azureml-opendatasets==1.21.0
  - azureml-automl-core==1.21.0
  - azureml-automl-runtime==1.21.0
  - azureml-train-automl-client==1.21.0
  - azureml-train-automl-runtime==1.21.0.post1
  - azureml-train-automl==1.21.0
  - azureml-train==1.21.0
  - azureml-sdk==1.21.0
  - azureml-interpret==1.21.0
  - azureml-tensorboard==1.21.0
  - azureml-mlflow==1.21.0
  - mlflow
  - sklearn-pandas
- pandas
- numpy
- tqdm
- scikit-learn
- matplotlib
name: azureml_df6ad66e80d4bc0030b6d046a4e46427



In [49]:
# If one needs to add packages
for pip_package in ["scikit-learn"]:
    myenv.python.conda_dependencies.add_pip_package(pip_package)
for pip_package in ["xgboost"]:
    myenv.python.conda_dependencies.add_pip_package(pip_package)    

In [50]:
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

In [51]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1,
                                                      enable_app_insights=True)

service = Model.deploy(
    workspace = ws, 
    name = 'titanic-attempt3', 
    models = [model], 
    inference_config = inference_config, 
    deployment_config = deployment_config)

service.wait_for_deployment(True)

print(service.state)
print("scoring URI: " + service.scoring_uri)
print(servide.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 9c2b1010-367e-48b8-99d1-6d6ec04d62cd
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'xgboost', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'xgboost', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 9c2b1010-367e-48b8-99d1-6d6ec04d62cd
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'xgboost', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'xgboost', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 9c2b1010-367e-48b8-99d1-6d6ec04d62cd\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"AciDeploymentFailed\",

TODO: In the cell below, send a request to the web service you deployed to test it.

In [30]:
scoring_uri = ''
key = ''

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "PassengerId": 1709,
            "Pclass": 1,
            "Sex": 0,
            "SibSp": 1,
            "Parch": 0,
            "Fare": 36.05,
            "FamilySize": 1,
            "IsAlone": 0,
            "y": 0
          },
          {
            "PassengerId": 1807,
            "Pclass": 3,
            "Sex":1,
            "SibSp":0,
            "Parch":0,
            "Fare": 72.08,
            "FamilySize": 1,
            "IsAlone":1,
            "y":1
          }
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)
    
# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

TODO: In the cell below, print the logs of the web service and delete the service

In [28]:
# standout suggestion
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

WebserviceException: WebserviceException:
	Message: WebserviceNotFound: Webservice with name Titanic_survival not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "WebserviceNotFound: Webservice with name Titanic_survival not found in provided workspace"
    }
}

In [ ]:
# delete service
service.delete()

In [ ]:
# Standout action: create and publish pipeline
ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

pipeline_run = experiment.submit(pipeline)

In [ ]:
RunDetails(pipeline_run).show()

In [ ]:
pipeline_run.wait_for_completion()

In [ ]:
# save the best model
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

In [ ]:
best_model.steps

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Titanic_survival", description="Titanic Survival pipeline", version="1.0")

published_pipeline

In [ ]:
interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [ ]:
rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)


In [ ]:
published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()